In [2]:
import random 
class Sentiment:
    Negative= 'NEGATIVE'
    Positive='POSITIVE'
    Neutral='NEUTRAL'
class Review:
    def __init__(self,text,stars):
        self.text=text
        self.stars=stars
        self.sentiment=self.get_sentiment()
    def get_sentiment(self):
        if self.stars<=2:
            return Sentiment.Negative
        elif self.stars==3:
            return Sentiment.Neutral
        else:
            return Sentiment.Positive
class ReviewContainer:
    def __init__(self,reviews):
        self.reviews=reviews
    def get_text(self):
        return [x.text for x in self.reviews]
    def get_sentiment(self):
        return [x.sentiment for x in self.reviews]
    def evenly_distributed(self):
       negative=list(filter(lambda x:x.sentiment== Sentiment.Negative, self.reviews))
       positive=list(filter(lambda x:x.sentiment== Sentiment.Positive, self.reviews))
       positive_shrunk=positive[:len(negative)]
       self.reviews=negative+positive_shrunk
       random.shuffle(self.reviews)
       
            

In [3]:
import json
file_name='../scikit-learn/Books_small_10000.json'
with open (file_name) as f:
    reviews=[]
    for line in f:
      review=json.loads(line)
      reviews.append(Review(review['reviewText'],review['overall']))
reviews[5].sentiment

'POSITIVE'

In [4]:
from sklearn.model_selection import train_test_split
training,testing=train_test_split(reviews,test_size=0.33, random_state =42)
train_cont=ReviewContainer(training)
test_cont=ReviewContainer(testing)
train_cont.evenly_distributed()
test_cont.evenly_distributed()
train_x=train_cont.get_text()
train_y=train_cont.get_sentiment()
test_x=test_cont.get_text()
test_y=test_cont.get_sentiment()
print(train_y.count(Sentiment.Negative))
print(train_y.count(Sentiment.Positive))
print(test_y.count(Sentiment.Negative))
print(test_y.count(Sentiment.Positive))




436
436
208
208


### Bags of words

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer =CountVectorizer()
train_x_vectors=vectorizer.fit_transform(train_x)
test_x_vectors=vectorizer.fit_transform(test_x)
# print(train_x_vectors[0].toarray())
test_x_vectors[0]

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 36 stored elements and shape (1, 5873)>

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle

# Fit and save the vectorizer
vectorizer = TfidfVectorizer()
train_x_vectors = vectorizer.fit_transform(train_x)
with open('vectorizer.pkl', 'wb') as f:
    pickle.dump(vectorizer, f)

# Load and transform the test data
with open('vectorizer.pkl', 'rb') as f:
    vectorizer = pickle.load(f)
test_x_vectors = vectorizer.transform(test_x)


In [7]:
from sklearn import svm
clf_svm=svm.SVC(kernel='linear')
clf_svm.fit(train_x_vectors, train_y)
clf_svm.predict(test_x_vectors[4])




array(['NEGATIVE'], dtype='<U8')

### Decision Tree

In [8]:
from sklearn.tree import DecisionTreeClassifier
clf_dec=DecisionTreeClassifier()
clf_dec.fit(train_x_vectors, train_y)
clf_dec.predict(test_x_vectors[4])


array(['NEGATIVE'], dtype='<U8')

In [9]:
from sklearn.linear_model import LogisticRegression
clf_log=LogisticRegression()
clf_log.fit(train_x_vectors, train_y)
clf_log.predict(test_x_vectors[4])

array(['NEGATIVE'], dtype='<U8')

### Evaluation

In [10]:
# Mean Accuracy
# clf_svm.score(train_x_vectors, train_y)
clf_dec.score(train_x_vectors, train_y)
# clf_log.score(train_x_vectors, train_y)

1.0

In [24]:
# F1-scores
from sklearn.metrics import f1_score
f1_score(test_y, clf_svm.predict(test_x_vectors),average=None,labels=[Sentiment.Negative, Sentiment.Positive, Sentiment.Neutral ],zero_division=0)
f1_score(test_y, clf_log.predict(test_x_vectors),average=None,labels=[Sentiment.Negative, Sentiment.Positive, Sentiment.Neutral ],zero_division=0)
f1_score(test_y, clf_dec.predict(test_x_vectors),average=None,labels=[Sentiment.Negative, Sentiment.Positive, Sentiment.Neutral ],zero_division=0)


array([0.63700234, 0.61728395, 0.        ])

In [12]:
train_y.count(Sentiment.Positive)
train_y.count(Sentiment.Negative)

436

In [13]:
test_set=["This book wasnt upto the mark! .I hated the protagonist's possesiveness and pevertness"]

new_tet_set=vectorizer.transform(test_set)
clf_svm.predict(new_test_set)

array(['NEGATIVE'], dtype='<U8')

Tuning Our model with grid search

In [25]:
from sklearn.model_selection import GridSearchCV

parameters = {'kernel':('linear', 'rbf'), 'C':[1,4,5, 10]}
svc=svm.SVC()
clf=GridSearchCV(svc,parameters,cv=5)
clf.fit(train_x_vectors,train_y)
f1_score(test_y, clf_svm.predict(test_x_vectors),average=None,labels=[Sentiment.Negative, Sentiment.Positive, Sentiment.Neutral ],zero_division=0)


array([0.80952381, 0.80582524, 0.        ])

Saving a model


In [15]:
import pickle

In [20]:
#writing  our model into a file
with open("../scikit-learn/models.pkl","wb") as f:
    pickle.dump(clf,f)
    

In [23]:
#prediction by our done from that file (reading)
with open("../scikit-learn/models.pkl","rb") as f:
    loaded_clf=pickle.load(f)
print(test_x[0])
loaded_clf.predict(test_x_vectors[0])


.are for the fact that I kept reading ,but sorry it just didn't work for me. I found it very bleak all the way through. Having said that it is smoothly written and if you like this who done it genre you may just relate to this.


array(['NEGATIVE'], dtype='<U8')